In [1]:
from yaml import safe_load
import pandas as pd
from pathlib import Path
import shutil

In [3]:
df = pd.read_parquet('/media/ts/SSD_ubuntu/datasets/MACS/annot_train.parquet')

In [4]:
df

,filename,captions,tags,target
0,train/airport-barcelona-0-0-a.flac,"[a person whistling and singing, people are ta...","[music, adults_talking, footsteps]",airport
1,train/airport-barcelona-0-10-a.flac,[whistling plates and cutlery clinging in a ca...,"[announcement_jingle, adults_talking, footstep...",airport
2,train/airport-barcelona-0-12-a.flac,[distant adults talking some package unfolding...,"[adults_talking, footsteps]",airport
3,train/airport-barcelona-0-14-a.flac,"[people are walking and talking, footsteps and...","[adults_talking, footsteps]",airport
4,train/airport-barcelona-0-3-a.flac,[people talking and moving whistling sound in ...,"[adults_talking, children_voices, footsteps]",airport
...,...,...,...,...
3336,train/public_square-vienna-253-7600-a.flac,"[people talking while the baby cries, adults t...","[traffic_noise, adults_talking, children_voice...",public_square
3337,train/public_square-vienna-253-7601-a.flac,[a man talking some other person talking quiet...,"[footsteps, adults_talking, birds_singing]",public_square
3338,train/public_square-vienna-253-7602-a.flac,[far away traffic noise motorcycle birds and f...,"[traffic_noise, adults_talking, birds_singing]",public_square
3339,train/public_square-vienna-253-7604-a.flac,[sound of the group of people joyfully talking...,"[ , adults_talking, children_voices, footsteps]",public_square


In [2]:
root = Path("/media/ts/SSD_ubuntu/datasets/MACS/")
with open(root / 'MACS.yaml', 'r') as f:
    data = safe_load(f)

In [3]:
df = pd.json_normalize(data['files'], 'annotations', ['filename'])
df

,annotator_id,sentence,tags,filename
0,233,a person whistling and singing,"[adults_talking, music]",airport-barcelona-0-0-a.wav
1,105,people are talking whistling and singing,[adults_talking],airport-barcelona-0-0-a.wav
2,332,whistling and singing many people talking,"[adults_talking, footsteps]",airport-barcelona-0-0-a.wav
3,150,people talking in the background a person whis...,"[music, adults_talking]",airport-barcelona-0-0-a.wav
4,251,whistling plates and cutlery clinging in a car...,[adults_talking],airport-barcelona-0-10-a.wav
...,...,...,...,...
17270,104,men and women are chatting with each other,[adults_talking],public_square-vienna-253-7604-a.wav
17271,364,many loud conversations are heard nearby,[adults_talking],public_square-vienna-253-7604-a.wav
17272,197,adults and children talking traffic noises,"[traffic_noise, adults_talking, children_voices]",public_square-vienna-253-7605-a.wav
17273,262,adults chatting at a place nearby,[adults_talking],public_square-vienna-253-7605-a.wav


In [4]:
tau_meta = (pd.read_csv(root / 'TAU-urban-acoustic-scenes-2019-development/meta.csv', sep='\t')
            .drop(columns=['identifier', 'source_label'])
            .rename(columns={'scene_label': 'target'})
            )
tau_meta.filename = tau_meta.filename.apply(lambda f: Path(f).with_suffix('.flac').name)
tau_meta.target = tau_meta.target.astype('category')
tau_meta = tau_meta[tau_meta.filename.apply(lambda f: (root / 'audio' / f).exists())]
tau_meta.to_parquet(root / 'TAU-urban-acoustic-scenes-2019-annot.parquet', index=False)
tau_meta

,filename,target
0,airport-lisbon-1000-40000-a.flac,airport
1,bus-lyon-1001-40001-a.flac,bus
2,shopping_mall-lisbon-1002-40002-a.flac,shopping_mall
3,street_pedestrian-lyon-1003-40003-a.flac,street_pedestrian
4,street_pedestrian-lisbon-1004-40004-a.flac,street_pedestrian
...,...,...
14395,tram-vienna-285-8635-a.flac,tram
14396,tram-vienna-285-8636-a.flac,tram
14397,tram-vienna-285-8637-a.flac,tram
14398,tram-vienna-285-8638-a.flac,tram


In [5]:
df_clean = (df
            .groupby('filename', as_index=False)
            ['sentence', 'tags']
            .agg({'sentence': list, 'tags': lambda x: set([item for sublist in x for item in sublist])})
            .rename(columns={'sentence': 'captions'})
            )
df_clean.filename = df_clean.filename.apply(lambda f: Path(f).with_suffix('.flac').name)
df_clean = df_clean.merge(tau_meta, on='filename')
df_clean.target = df_clean.target.cat.remove_unused_categories()
df_clean = df_clean[df_clean.filename.apply(lambda f: (root / 'audio' / f).exists())]
df_clean.to_parquet(root / 'MACS_annot.parquet', index=False)
df_clean

/tmp/ipykernel_195302/4014960439.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_clean = (df


,filename,captions,tags,target
0,airport-barcelona-0-0-a.flac,"[a person whistling and singing, people are ta...","{music, adults_talking, footsteps}",airport
1,airport-barcelona-0-10-a.flac,[whistling plates and cutlery clinging in a ca...,"{announcement_jingle, adults_talking, footstep...",airport
2,airport-barcelona-0-11-a.flac,"[whistling footsteps and adults talking, adult...","{adults_talking, footsteps}",airport
3,airport-barcelona-0-12-a.flac,[distant adults talking some package unfolding...,"{adults_talking, footsteps}",airport
4,airport-barcelona-0-13-a.flac,[a lot of people are talking and walking aroun...,"{adults_talking, children_voices, footsteps}",airport
...,...,...,...,...
3925,public_square-vienna-253-7600-a.flac,"[people talking while the baby cries, adults t...","{traffic_noise, adults_talking, children_voice...",public_square
3926,public_square-vienna-253-7601-a.flac,[a man talking some other person talking quiet...,"{footsteps, adults_talking, birds_singing}",public_square
3927,public_square-vienna-253-7602-a.flac,[far away traffic noise motorcycle birds and f...,"{traffic_noise, adults_talking, birds_singing}",public_square
3928,public_square-vienna-253-7604-a.flac,[sound of the group of people joyfully talking...,"{ , adults_talking, children_voices, footsteps}",public_square


In [9]:
x = df_clean.captions.apply(len).value_counts()

In [16]:
x

5    2188
4    1120
3     611
2      11
Name: captions, dtype: int64

In [7]:
macs_dir = root / 'macs_audio'
macs_dir.mkdir()
for _, row in df_clean.iterrows():
    shutil.copy(root / 'audio' / row['filename'], macs_dir / row['filename'])

In [33]:
df = pd.read_parquet('/media/ts/SSD_ubuntu/datasets/MACS/MACS_annot.parquet')
df

,filename,captions,tags,target
0,airport-barcelona-0-0-a.flac,"[a person whistling and singing, people are ta...","[music, adults_talking, footsteps]",airport
1,airport-barcelona-0-10-a.flac,[whistling plates and cutlery clinging in a ca...,"[announcement_jingle, adults_talking, footstep...",airport
2,airport-barcelona-0-11-a.flac,"[whistling footsteps and adults talking, adult...","[adults_talking, footsteps]",airport
3,airport-barcelona-0-12-a.flac,[distant adults talking some package unfolding...,"[adults_talking, footsteps]",airport
4,airport-barcelona-0-13-a.flac,[a lot of people are talking and walking aroun...,"[adults_talking, children_voices, footsteps]",airport
...,...,...,...,...
3925,public_square-vienna-253-7600-a.flac,"[people talking while the baby cries, adults t...","[traffic_noise, adults_talking, children_voice...",public_square
3926,public_square-vienna-253-7601-a.flac,[a man talking some other person talking quiet...,"[footsteps, adults_talking, birds_singing]",public_square
3927,public_square-vienna-253-7602-a.flac,[far away traffic noise motorcycle birds and f...,"[traffic_noise, adults_talking, birds_singing]",public_square
3928,public_square-vienna-253-7604-a.flac,[sound of the group of people joyfully talking...,"[ , adults_talking, children_voices, footsteps]",public_square


In [34]:
df['freq'] = 1. / df.groupby('target')['target'].transform('count')
sampledf = df.sample(n=int(0.15 * len(df)), weights=df.freq)
sampledf.target.value_counts()

park             197
airport          196
public_square    196
Name: target, dtype: int64

In [35]:
sampledf = sampledf.drop(columns=['freq'])
sampledf.to_parquet('/media/ts/SSD_ubuntu/datasets/MACS/annot_val.parquet', index=False)
sampledf

,filename,captions,tags,target
1451,park-helsinki-242-7209-a.flac,[far away talking nearby footsteps and far awa...,"[children_voices, dog_barking, footsteps, adul...",park
769,airport-milan-1172-45710-a.flac,[the person says something while people are ma...,"[traffic_noise, adults_talking, children_voice...",airport
31,airport-barcelona-1-37-a.flac,[man talking while other people are walking in...,"[traffic_noise, adults_talking, children_voice...",airport
3492,public_square-paris-118-3470-a.flac,"[distant teenager voices in the middle, footst...","[adults_talking, children_voices, footsteps, b...",public_square
3551,public_square-prague-1027-42405-a.flac,[a whine of a motor and the air rushing over t...,"[ , traffic_noise]",public_square
...,...,...,...,...
2403,park-stockholm-102-2911-a.flac,[footsteps on gravelly soil with electrical st...,"[adults_talking, children_voices, footsteps]",park
243,airport-helsinki-4-216-a.flac,[talking in the distance squeaky noises static...,[adults_talking],airport
72,airport-barcelona-1-85-a.flac,[announcement jingle and somebody speak while ...,"[announcement_jingle, adults_talking, children...",airport
1824,park-lyon-1012-40014-a.flac,"[sound of a traffic road, very calm surroundin...","[traffic_noise, ]",park


In [36]:
df = df.drop(index=sampledf.index, columns=['freq'])
df.to_parquet('/media/ts/SSD_ubuntu/datasets/MACS/annot_train.parquet', index=False)
df

,filename,captions,tags,target
0,airport-barcelona-0-0-a.flac,"[a person whistling and singing, people are ta...","[music, adults_talking, footsteps]",airport
1,airport-barcelona-0-10-a.flac,[whistling plates and cutlery clinging in a ca...,"[announcement_jingle, adults_talking, footstep...",airport
3,airport-barcelona-0-12-a.flac,[distant adults talking some package unfolding...,"[adults_talking, footsteps]",airport
5,airport-barcelona-0-14-a.flac,"[people are walking and talking, footsteps and...","[adults_talking, footsteps]",airport
7,airport-barcelona-0-3-a.flac,[people talking and moving whistling sound in ...,"[adults_talking, children_voices, footsteps]",airport
...,...,...,...,...
3925,public_square-vienna-253-7600-a.flac,"[people talking while the baby cries, adults t...","[traffic_noise, adults_talking, children_voice...",public_square
3926,public_square-vienna-253-7601-a.flac,[a man talking some other person talking quiet...,"[footsteps, adults_talking, birds_singing]",public_square
3927,public_square-vienna-253-7602-a.flac,[far away traffic noise motorcycle birds and f...,"[traffic_noise, adults_talking, birds_singing]",public_square
3928,public_square-vienna-253-7604-a.flac,[sound of the group of people joyfully talking...,"[ , adults_talking, children_voices, footsteps]",public_square


In [55]:
val_df = pd.read_parquet('/media/ts/SSD_ubuntu/datasets/MACS/macs/annot_train.parquet')
val_df.filename = val_df.filename.apply(lambda x: 'train/' + x)
val_df.to_parquet('/media/ts/SSD_ubuntu/datasets/MACS/annot_train.parquet',
              index=False)
val_df

,filename,captions,tags,target
0,train/airport-barcelona-0-0-a.flac,"[a person whistling and singing, people are ta...","[music, adults_talking, footsteps]",airport
1,train/airport-barcelona-0-10-a.flac,[whistling plates and cutlery clinging in a ca...,"[announcement_jingle, adults_talking, footstep...",airport
2,train/airport-barcelona-0-12-a.flac,[distant adults talking some package unfolding...,"[adults_talking, footsteps]",airport
3,train/airport-barcelona-0-14-a.flac,"[people are walking and talking, footsteps and...","[adults_talking, footsteps]",airport
4,train/airport-barcelona-0-3-a.flac,[people talking and moving whistling sound in ...,"[adults_talking, children_voices, footsteps]",airport
...,...,...,...,...
3336,train/public_square-vienna-253-7600-a.flac,"[people talking while the baby cries, adults t...","[traffic_noise, adults_talking, children_voice...",public_square
3337,train/public_square-vienna-253-7601-a.flac,[a man talking some other person talking quiet...,"[footsteps, adults_talking, birds_singing]",public_square
3338,train/public_square-vienna-253-7602-a.flac,[far away traffic noise motorcycle birds and f...,"[traffic_noise, adults_talking, birds_singing]",public_square
3339,train/public_square-vienna-253-7604-a.flac,[sound of the group of people joyfully talking...,"[ , adults_talking, children_voices, footsteps]",public_square
